In [14]:
import os
import faiss
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from app.database.models import Book, User, Badge
from app.database.crud import create_book, get_books
from sqlalchemy import text
from dotenv import load_dotenv

load_dotenv()

MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_PORT = os.getenv('MYSQL_PORT')
MYSQL_DB = os.getenv('MYSQL_DB')

# 데이터베이스 연결 설정
DATABASE_URL = f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# 데이터베이스 세션 생성
mysql_db = SessionLocal()

In [19]:
POSTGRES_USER= os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DB = os.getenv("POSTGRES_DB")

# 데이터베이스 연결 설정
DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# 데이터베이스 세션 생성
postgresql_db = SessionLocal()

In [3]:
result = mysql_db.execute(text("SHOW CREATE TABLE badges"))
table_structure = result.fetchone()[1]  # 결과에서 BADGE TABLE 구문 가져오기
print(table_structure)  # 테이블 생성 SQL 출력

CREATE TABLE `badges` (
  `badge_id` bigint NOT NULL AUTO_INCREMENT,
  `user_id` varchar(64) NOT NULL,
  `book_id` bigint NOT NULL,
  `badge_image` varchar(2048) NOT NULL,
  `created_at` timestamp NOT NULL DEFAULT (now()),
  PRIMARY KEY (`badge_id`),
  KEY `user_id` (`user_id`),
  KEY `book_id` (`book_id`),
  CONSTRAINT `badges_ibfk_1` FOREIGN KEY (`user_id`) REFERENCES `users` (`user_id`),
  CONSTRAINT `badges_ibfk_2` FOREIGN KEY (`book_id`) REFERENCES `books` (`book_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


In [5]:
result = mysql_db.execute(text("SHOW CREATE TABLE users"))
table_structure = result.fetchone()[1]  # 결과에서 BADGE TABLE 구문 가져오기
print(table_structure)  # 테이블 생성 SQL 출력

CREATE TABLE `users` (
  `user_id` varchar(64) NOT NULL,
  `birth_year` varchar(4) NOT NULL,
  `gender` enum('M','F') NOT NULL,
  PRIMARY KEY (`user_id`),
  CONSTRAINT `chk_birth_year_format` CHECK (((length(`birth_year`) = 4) and regexp_like(`birth_year`,_utf8mb4'^[0-9]{4}$')))
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


In [39]:
mysql_db.rollback()

result = mysql_db.execute(text("SHOW CREATE TABLE books"))
table_structure = result.fetchone()[1]  # 결과에서 BADGE TABLE 구문 가져오기
print(table_structure)  # 테이블 생성 SQL 출력

CREATE TABLE `books` (
  `book_id` bigint NOT NULL AUTO_INCREMENT,
  `title` varchar(1024) NOT NULL,
  `author` varchar(1024) DEFAULT NULL,
  `publisher` varchar(255) DEFAULT NULL,
  `pubdate` date DEFAULT NULL,
  `isbn` varchar(20) DEFAULT NULL,
  `description` text,
  `image` varchar(2048) DEFAULT NULL,
  PRIMARY KEY (`book_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


In [38]:
mysql_db.rollback()
postgresql_db.rollback()

In [30]:
result = mysql_db.execute(text("SHOW CREATE TABLE recommended_books"))
table_structure = result.fetchone()[1]  # 결과에서 BADGE TABLE 구문 가져오기
print(table_structure)  # 테이블 생성 SQL 출력

CREATE TABLE `recommended_books` (
  `recommendation_id` bigint NOT NULL AUTO_INCREMENT,
  `book_id` bigint NOT NULL,
  `user_id` varchar(64) NOT NULL,
  `session_id` varchar(255) NOT NULL,
  `recommended_at` timestamp NOT NULL DEFAULT (now()),
  `finished_at` timestamp NULL DEFAULT NULL,
  PRIMARY KEY (`recommendation_id`),
  KEY `book_id` (`book_id`),
  KEY `user_id` (`user_id`),
  KEY `session_id` (`session_id`),
  CONSTRAINT `recommended_books_ibfk_1` FOREIGN KEY (`book_id`) REFERENCES `books` (`book_id`),
  CONSTRAINT `recommended_books_ibfk_2` FOREIGN KEY (`user_id`) REFERENCES `users` (`user_id`),
  CONSTRAINT `recommended_books_ibfk_3` FOREIGN KEY (`session_id`) REFERENCES `sessions` (`session_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


In [23]:
query = """
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'user_questions'
"""

result = postgresql_db.execute(text(query))
columns = result.fetchall()

for column in columns:
    print(column)

('question_id', 'bigint')
('question_text', 'jsonb')
('created_at', 'timestamp without time zone')
('user_id', 'character varying')
('session_id', 'character varying')


In [26]:
query = """
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'clova_answers'
"""

result = postgresql_db.execute(text(query))
columns = result.fetchall()

for column in columns:
    print(column)

('answer_id', 'bigint')
('answer_text', 'jsonb')
('created_at', 'timestamp without time zone')
('user_id', 'character varying')
('session_id', 'character varying')


In [5]:
def alter_badge_table(db):
    try:
        # `badge_audio_url` 컬럼 삭제
        db.execute(text("ALTER TABLE badges DROP COLUMN badge_audio_url"))

        db.commit()
        print("🚀 [수정 완료] `badge_audio_url` 컬럼이 삭제되었습니다!")
    except Exception as e:
        db.rollback()
        print(f"❌ [오류 발생] {e}")

# 실행
alter_badge_table(mysql_db)

🚀 [수정 완료] `badge_audio_url` 컬럼이 삭제되었습니다!


In [10]:
def alter_users_table(db):
    try:
        # `name`, `phone_number`, `email` 컬럼 삭제
        db.execute(text("ALTER TABLE users DROP COLUMN name"))
        db.execute(text("ALTER TABLE users DROP COLUMN phone_number"))
        db.execute(text("ALTER TABLE users DROP COLUMN email"))

        db.commit()
        print("🚀 [수정 완료] `name`, `phone_number`, `email` 컬럼이 삭제되었습니다!")
    except Exception as e:
        db.rollback()
        print(f"❌ [오류 발생] {e}")

# 실행
alter_users_table(mysql_db)

❌ [오류 발생] (pymysql.err.OperationalError) (1091, "Can't DROP 'name'; check that column/key exists")
[SQL: ALTER TABLE users DROP COLUMN name]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [29]:
mysql_db. rollback()

def alter_recommended_books_table(db):
    try:
        # ✅ 올바른 테이블 이름으로 수정
        db.execute(text("ALTER TABLE recommended_books MODIFY COLUMN finished_at TIMESTAMP NULL DEFAULT NULL"))
        db.commit()
        print("🚀 [수정 완료]")
    except Exception as e:
        db.rollback()
        print(f"❌ [오류 발생] {e}")

# 실행
alter_recommended_books_table(mysql_db)


🚀 [수정 완료]


In [37]:
mysql_db.rollback()

def alter_books_table(db):
    try:
        # ✅ description 컬럼을 NULL 허용으로 변경
        db.execute(text("ALTER TABLE books MODIFY COLUMN description TEXT NULL"))
        db.commit()
        print("🚀 [수정 완료] description 컬럼이 NULL을 허용하도록 변경됨!")
    except Exception as e:
        db.rollback()
        print(f"❌ [오류 발생] {e}")

# 실행
alter_books_table(mysql_db)

KeyboardInterrupt: 

In [40]:
from sqlalchemy import text
from sqlalchemy.orm import Session

def truncate_tables(db: Session):
    """ 📌 books, recommended_books, sessions, user_questions, clova_answers 테이블 초기화 """
    try:
        db.rollback()  # 롤백 후 실행 (안전성 확보)

        db.execute(text("SET FOREIGN_KEY_CHECKS=0;"))  # 외래 키 체크 해제

        # 테이블 초기화
        db.execute(text("TRUNCATE TABLE books;"))
        db.execute(text("TRUNCATE TABLE recommended_books;"))
        db.execute(text("TRUNCATE TABLE sessions;"))
        db.execute(text("TRUNCATE TABLE user_questions;"))
        db.execute(text("TRUNCATE TABLE clova_answers;"))

        db.execute(text("SET FOREIGN_KEY_CHECKS=1;"))  # 외래 키 체크 활성화
        db.commit()

        print("🚀 [초기화 완료] 모든 테이블이 초기화되었습니다!")

    except Exception as e:
        db.rollback()
        if "Unknown table" in str(e):
            print("⚠️ [경고] 일부 테이블이 존재하지 않습니다. 테이블을 먼저 생성하세요.")
        else:
            print(f"❌ [초기화 오류] {e}")

# 실행
truncate_tables(mysql_db)

KeyboardInterrupt: 

In [42]:
mysql_db.rollback()